In [ ]:
#default_exp pdUtils

# pd Utils
utilities function for pandas dataframe and series

In [ ]:
#export
import pandas as pd
from hashlib import sha1
from nicHelper.dictUtil import saveStringToFile, loadStringFromFile
from pynamodb.constants import BINARY
from pynamodb.attributes import Attribute, UnicodeAttribute
from pynamodb.models import Model
from beartype import beartype
from s3bz.s3bz import S3
import os, logging

In [ ]:
logging.basicConfig(level = logging.WARNING)

In [ ]:
df = pd.DataFrame({'hello':[1,2,3,4,5,5]})
df

,hello
0,1
1,2
2,3
3,4
4,5
5,5


In [ ]:
from random import randint
from randstr import randstr
largeDf = pd.DataFrame({'hello':[randint(1,100000) for _ in range(1000)], 
                        'hello2':[randint(1,100000) for _ in range(1000)],
                        randstr(5):[randstr(30) for _ in range(1000)],
                        randstr(5):[randstr(30) for _ in range(1000)],
                        randstr(5):[randstr(30) for _ in range(1000)],
                        randstr(5):[randstr(30) for _ in range(1000)],
                        randstr(5):[randstr(30) for _ in range(1000)],
                        randstr(5):[randstr(30) for _ in range(1000)],
                       })

# getDfHash

In [ ]:
#export
from io import BytesIO
from typing import Callable

@beartype
def getDfHash(df:pd.DataFrame, 
              hashingAlgorithm: Callable = lambda x: sha1(x).hexdigest())->str:
  '''
    get a hash of a pandas dataframe
    this uses sha1 algorithm
    inputs:
      df: pd.DataFrame: a pandas dataframe
      hashingAlgoritm: callable: a hasing function which takes bytes input
    response:
      string hash
  '''
  f:BytesIO = BytesIO()
  df.to_feather(f)
  return hashingAlgorithm(f.read())


In [ ]:
%%timeit
getDfHash(largeDf)

2.04 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
import joblib
joblib.hash(largeDf)

15 ms ± 266 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
def testFeather(df):
  f:BytesIO = BytesIO()
  df.to_feather(f)
  
%timeit testFeather(largeDf)

1.89 ms ± 56.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Local cache and hash

In [ ]:
#export
def saveLocalCache( data:pd.DataFrame, path:str = '/tmp/cache', 
                   saveHash:bool = True, force:bool = True):
  '''
    save cache of the dataframe to local location
    data:pd.DataFrame: dataframe to save
    path: str: path to save cache
    saveHash: bool: whether to save the hash digest
  '''
  ##check cache
  if not force:
    localHash = loadLocalHash(f'{path}.hash')
    dataHash = getDfHash(data)
    if dataHash == localHash :
      logging.debug('hash is the latest, skipping')
      return True
  ##save hash
  if saveHash: 
    saveLocalHash(data, path=f'{path}.hash')
  # save cache
  logging.debug('saving cache')
  r =  data.to_feather(path)
  return r

def saveLocalHash( data:pd.DataFrame, path = '/tmp/cache.hash', force = False):
  dataHash = getDfHash(data)
  return saveStringToFile(dataHash,path)

def loadLocalCache( path = '/tmp/cache', throw = True):
  if not os.path.exists(path):
    if throw:
      raise Exception('cache doesnt exist')
  return pd.read_feather(path)
def loadLocalHash( path = '/tmp/cache.hash'):
  if not os.path.exists(path): raise Exception('hash doesnt exist')
  return loadStringFromFile(path)

In [ ]:
%time saveLocalCache(df,force = True)
%time saveLocalHash(df)
%time print(loadLocalHash())
%time loadLocalCache()

CPU times: user 0 ns, sys: 2.91 ms, total: 2.91 ms
Wall time: 5.12 ms
CPU times: user 1 ms, sys: 110 µs, total: 1.11 ms
Wall time: 897 µs
da39a3ee5e6b4b0d3255bfef95601890afd80709
CPU times: user 297 µs, sys: 33 µs, total: 330 µs
Wall time: 272 µs
CPU times: user 2.22 ms, sys: 0 ns, total: 2.22 ms
Wall time: 3.41 ms


,hello
0,1
1,2
2,3
3,4
4,5
5,5


## Remote cache and hash

In [ ]:
#export
def saveRemoteHash(data:pd.DataFrame, key='', bucket='', **kwargs):
  hashKey = f'{key}-hash'
  hashString = getDfHash(data)
  dictToSave= {'hash': hashString }
  print(f'hashKey is {hashKey}')
  print('saving hash to s3')
  S3.save(key=hashKey,objectToSave=dictToSave, bucket=bucket, **kwargs )
  print(f'saved hash {hashString}')
  

def saveRemoteCache(data:pd.DataFrame, key = '', 
                    bucket = '', localCachePath='/tmp/cache', localHashPath='/tmp/hash', **kwargs):
  
  saveLocalCache(data=data, path = localCachePath)
  saveLocalHash(data=data, path = localHashPath)
  saveRemoteHash(data=data, key = key, bucket=bucket)
  S3.saveFile(key=key, path=localCachePath, bucket=bucket, **kwargs)
  
def loadRemoteCache(key='', bucket='', **kwargs):
  path = '/tmp/tmpPath'
  S3.loadFile(key,path=path ,bucket=bucket, **kwargs)
  df = pd.read_feather(path)
  return df

def loadRemoteHash(key='', bucket='', **kwargs):
  hashKey = f'{key}-hash'
  print(f'loading hashkey {hashKey}')
  loadedHash= S3.load(hashKey,bucket=bucket, **kwargs).get('hash')
  print(f'loaded hash is {loadedHash}')
  return loadedHash

In [ ]:
#hide
testKey = 'testKey'
testBucket = 'villa-clipboard'
%time saveRemoteCache(df, key = testKey, bucket = testBucket)
print(loadRemoteHash(testKey, testBucket))
%timeit loadRemoteCache(key = testKey, bucket = testBucket)

hashKey is testKey-hash
saving hash to s3
saved hash da39a3ee5e6b4b0d3255bfef95601890afd80709
CPU times: user 81.3 ms, sys: 14.5 ms, total: 95.8 ms
Wall time: 240 ms
loading hashkey testKey-hash
loaded hash is da39a3ee5e6b4b0d3255bfef95601890afd80709
da39a3ee5e6b4b0d3255bfef95601890afd80709
125 ms ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## PynamoAttributes

In [ ]:
#export
class PandasDataFrameAttribute(Attribute):
  attr_type = BINARY
  def serialize(self, value: pd.DataFrame)->bin:
    bio = BytesIO()
    value.to_feather(bio)
    data:bin = bio.getvalue()
    return data
  def deserialize(self, value: bin)->pd.DataFrame:
    bio = BytesIo(bin)
    df: pd.DataFrame = pd.read_feather(bio)
    return df

In [ ]:
class Database(Model):
  class Meta:
    table_name = ''
    region = ''
    billing_mode='PAY_PER_REQUEST'
    
  brcode = UnicodeAttribute(hash_key=True, default = '')
  data = PandasDataFrameAttribute()
  
import sys
df = pd.DataFrame({'cprcode':['1234', '12345'], 'quantity':[123, 345]})
db = Database(brcode='1234', data = df)
db.data

,cprcode,quantity
0,1234,123
1,12345,345


In [ ]:
#export
# class PandasSeriesAttribute(Attribute):
#   attr_type = BINARY
#   def serialize(self, value: pd.Series)->bin:
#     bio = BytesIO()
#     df = s.to_frame()
#     value.to_feather(bio)
#     data:bin = bio.getvalue()
#     return data
#   def deserialize(self, value: bin)->pd.DataFrame:
#     bio = BytesIo(bin)
#     df: pd.DataFrame = pd.read_feather(bio)
#     return df

In [ ]:
#export
from nicHelper.schema import getTypes

In [ ]:
url = 'https://raw.githubusercontent.com/thanakijwanavit/villaMasterSchema/dev-manual/inventory/inventory.yaml'


inv = {
                  'iprcode': '0000009',
                  'brcode': '1000',
                  'ib_cf_qty': '50',
                  'new_ib_vs_stock_cv': '27',
                  'onlineflag': True
                }
getTypes(url)

{'iprcode': int,
 'brcode': int,
 'ib_cf_qty': int,
 'new_ib_vs_stock_cv': int,
 'onlineflag': bool}

In [ ]:
#export
def forceType(url:str, df:pd.DataFrame, defaultType=str)->pd.DataFrame:
  typeDict = getTypes(url)
  typeList = {col:typeDict.get(col) for col in df.columns}
  print(typeList)
  df = df.astype(typeList)
  print(df.dtypes)
  return df

In [ ]:
forceType(url, pd.DataFrame([inv]))

{'iprcode': <class 'int'>, 'brcode': <class 'int'>, 'ib_cf_qty': <class 'int'>, 'new_ib_vs_stock_cv': <class 'int'>, 'onlineflag': <class 'bool'>}
iprcode               int64
brcode                int64
ib_cf_qty             int64
new_ib_vs_stock_cv    int64
onlineflag             bool
dtype: object


,iprcode,brcode,ib_cf_qty,new_ib_vs_stock_cv,onlineflag
0,9,1000,50,27,True
